<a href="https://colab.research.google.com/github/vkjadon/openai/blob/main/04oai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We can load the api key from `.env` file by importing `load_dotenv` method of `dotenv` package. We can install `dotenv` package as

> `pip install python-dotenv`

> `from dotenv import load_dotenv, find_dotenv`

`find_dotenv()` : This function searches up the directory tree from your current working directory until it locates the .env file.

`load_dotenv()` : This function reads the variables from the found .env file and loads them into your Python process's environment variables (specifically, os.environ).

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

client = OpenAI()

In case you are running the code on Google Colab, add your "OPENAI_API_KEY" key in the `Secrets` of the colab environment and call instantiate the `OpenAI` object as below

In [4]:
!pip install -q faiss-cpu pypdf tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 19.6 MB/s eta 0:00:00


In [5]:
from openai import OpenAI
from google.colab import userdata
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [2]:
from google.colab import files

uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving download.pdf to download.pdf


In [6]:
from pypdf import PdfReader

reader = PdfReader(pdf_path)

text = ""
for page in reader.pages:
    text += page.extract_text()

print(text[:1000])  # preview


Different types of AI
Estimated time: 10 minutes
Introduction
Artiﬁcial intelligence (AI) encompasses a range of systems designed to mimic, enhance, or exceed human capabilities. AI can be categorized based on its capabilitiesand functionalities. Understanding these types and their capabilities highlights the diverse applications and potential of AI technologies.
Objectives
After completing this reading, you will be able to:
Explain the types of artiﬁcial intelligence based on their functionalities.
Explore the capabilities of each type of artiﬁcial intelligence.
AI types
1. Diagnostic/descriptive AI
Diagnostic or descriptive AI focuses on assessing the correctness of behavior by analyzing historical data to understand what happened and why. This type ofAI is instrumental in identifying patterns and trends, performing comparative analyses, and conducting root cause analyses.
Capabilities:
Scenario planning: Helps in creating different future scenarios based on historical data.
Pattern/

In [7]:
def chunk_text(text, chunk_size=800, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap
    return chunks

In [8]:
chunks = chunk_text(text)
print("Total chunks:", len(chunks))

Total chunks: 11


In [9]:
import numpy as np

In [10]:
def get_embeddings(texts):
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=texts
    )
    return np.array([e.embedding for e in response.data])

In [11]:
chunk_embeddings = get_embeddings(chunks)
print(chunk_embeddings.shape)

(11, 1536)


FAISS (Facebook AI Similarity Search) is an open-source library for efficient similarity search and clustering of dense, high-dimensional vectors. It is used to quickly find items in massive datasets that are "similar" to a given query item, solving performance bottlenecks that traditional databases face when dealing with unstructured data like images or text embeddings.

In [12]:
import faiss

dimension = chunk_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(chunk_embeddings)

print("FAISS index size:", index.ntotal)

FAISS index size: 11


In [13]:
def retrieve_chunks(query, k=3):
    query_embedding = get_embeddings([query])
    distances, indices = index.search(query_embedding, k)
    return [chunks[i] for i in indices[0]]

In [14]:
def build_context(query):
    retrieved = retrieve_chunks(query)
    return "\n\n".join(retrieved)

In [17]:
def ask_rag(question):
    context = build_context(question)

    response = client.responses.create(
        model="gpt-4.1-mini",
        input=[
            {
                "role": "system",
                "content": "You are a helpful teacher."
            },
            {
                "role": "developer",
                "content": (
                    "Answer ONLY using the provided context. "
                    "If the answer is not present, say 'I do not know.'"
                )
            },
            {
                "role": "user",
                "content": f"""
Context:
{context}

Question:
{question}
"""
            }
        ]
    )
    return response.output_text


In [19]:
question = "What are different types of artiﬁcial intelligence"
answer = ask_rag(question)

print(answer)


The different types of artificial intelligence include:

1. Diagnostic/descriptive AI: Focuses on assessing the correctness of behavior by analyzing historical data to understand what happened and why.
2. Predictive AI: Concerned with forecasting future outcomes based on historical and current data.
3. Prescriptive AI: Focuses on determining the optimal course of action by providing recommendations based on data analysis.
4. Generative/cognitive AI: Involved in producing various types of content, such as code, articles, images, and more.
5. Reactive AI: Designed to respond to specific inputs with predetermined responses.
6. Limited memory AI: Has the ability to use past experiences to inform current decisions.
7. Theory of Mind AI: Advanced type of AI that aims to understand human emotions, beliefs, and intentions.
8. Self-aware AI: The most advanced form of AI, which has its own consciousness and self-awareness.
9. Narrow AI (Weak AI): Designed to perform a specific task or a limited 